# Reconciling alpha in statsmodels and sklearn ridge regression

This analysis by Paul Zivich (https://sph.unc.edu/adv_profile/paul-zivich/) explains how to get the same results of ridge regression from statsmodels and sklearn. The difference is that sklearn's Ridge function internally scales the input of the 'alpha' regularization term during excecution as alpha / n_samples where n_samples is the number of samples, compared with statsmodels which does not apply this scaling of the regularization parameter during execution. You can have the ridge implementations match if you re-scale the sklearn input alpha = alpha / n_samples for statsmodels. Note that this rescaling of alpha only applies to ridge regression. The sklearn and statsmodels results for Lasso regression using exactly the same alpha values for input without rescaling.

Here is a link to the original post of this analysis by Paul Zivich on stackoverflow.com:  

https://stackoverflow.com/questions/72260808/mismatch-between-statsmodels-and-sklearn-ridge-regression

While comparing statsmodels and sklearn, Paul found that the two libraries result in different output for ridge regression. Below is an simple example of the difference

In [12]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
from sklearn.linear_model import Lasso, Ridge

np.random.seed(142131)

n = 500
d = pd.DataFrame()
d['A'] = np.random.normal(size=n)
d['B'] = d['A'] + np.random.normal(scale=0.25, size=n)
d['C'] = np.random.normal(size=n)
d['D'] = np.random.normal(size=n)
d['intercept'] = 1
d['Y'] = 5 - 2*d['A'] + 1*d['D'] + np.random.normal(size=n)

y = np.asarray(d['Y'])
X = np.asarray(d[['intercept', 'A', 'B', 'C', 'D']])

First, using sklearn and ridge:

In [13]:
alpha_sklearn = 1
ridge = Ridge(alpha=alpha_sklearn, fit_intercept=True)
ridge.fit(X=np.asarray(d[['A', 'B', 'C', 'D']]), y=y)
print('ridge params from sklearn intercept and coefs: \n',ridge.intercept_, ridge.coef_)

ridge params from sklearn intercept and coefs: 
 4.997208595888691 [-2.00968258  0.03363013 -0.02144874  1.02895154]


Next, statsmodels and OLS.fit_regularized:

In [14]:
alpha_statsmodels = np.array([0, 1., 1., 1., 1.])
ols = sm.OLS(y, X).fit_regularized(L1_wt=0., alpha=alpha_statsmodels)
print('ridge params from statsmodels: \n',ols.params)

ridge params from statsmodels: 
 [ 5.01623298e+00 -6.91643749e-01 -6.39008772e-01  1.55825435e-03
  5.51575433e-01]


which outputs [5.01623, -0.69164, -0.63901, 0.00156, 0.55158]. However, since these both are implementing ridge regression, Paul expected them to be the same.

Note, that neither of these penalize the intercept term (Paul checked that as a possible potential difference). Paul found that statsmodels and sklearn provide the same output for LASSO regression. Below is a demonstration with the previous data:

In [15]:
# sklearn LASSO
alpha_sklearn = 0.5
lasso = Lasso(alpha=alpha_sklearn, fit_intercept=True)
lasso.fit(X=np.asarray(d[['A', 'B', 'C', 'D']]), y=y)
print('lasso params from sklearn intercept and coefs: \n',lasso.intercept_, lasso.coef_)

# statsmodels LASSO
alpha_statsmodels = np.array([0, 0.5, 0.5, 0.5, 0.5])
ols = sm.OLS(y, X).fit_regularized(L1_wt=1., alpha=alpha_statsmodels)
print('lasso params from statsmodels: \n',ols.params)

lasso params from sklearn intercept and coefs: 
 5.014649977131442 [-1.5183174  -0.          0.          0.57799164]
lasso params from statsmodels: 
 [ 5.01464998 -1.51831729  0.          0.          0.57799166]


which both output [5.01465, -1.51832, 0., 0., 0.57799].

So Paul's question is why do the estimated coefficients for ridge regression differ across implementations in sklearn and statsmodels?

After digging around a little more, Paul discovered the answer by trial and error as to why the statsmodels and sklearn ridge regression results differ. The difference is that sklearn's Ridge scales the regularization term as alpha_scaled = alpha_input / n where n is the number of observations and alpha_input is the input argument values of alpha used with sklearn. statsmodels does not apply this scaling of the regularization parameter. You can have the statsmodels and sklearn ridge implementations match if you re-scale the regularizaiton parameter used for input to sklearn when you prepare the input required for statsmodels.

In other words, if you use the following input values of alpha for sklearn:

alpha_sklearn = 1

then you would need to use the following input of alpha=alpha_scaled when using statsmodels to get the same result:

alpha_statsmodels = alpha_sklearn / n_samples

where n_samples is the number of samples (n_samples = X.shape[0]).

Using Paul's posted example, here is how you would have the output of ridge regression parameters match between the statsmodels and sklearn:

In [16]:
# sklearn 
# NOTE: there is no difference from above
alpha_sklearn = 1
ridge = Ridge(alpha=alpha_sklearn, fit_intercept=True)
ridge.fit(X=np.asarray(d[['A', 'B', 'C', 'D']]), y=y)
print('ridge params from sklearn intercept and coefs: \n',ridge.intercept_, ridge.coef_)

# statsmodels
# NOTE: going to re-scale the regularization parameter based on n observations
n_samples = X.shape[0]
alpha_statsmodels = np.array([0, 1., 1., 1., 1.]) / n_samples  # scaling of alpha by n
ols = sm.OLS(y, X).fit_regularized(L1_wt=0., alpha=alpha_statsmodels)
print('ridge params from statsmodels with alpha=alpha/n: \n',ols.params)

ridge params from sklearn intercept and coefs: 
 4.997208595888691 [-2.00968258  0.03363013 -0.02144874  1.02895154]
ridge params from statsmodels with alpha=alpha/n: 
 [ 4.9972086  -2.00968258  0.03363013 -0.02144874  1.02895154]


Now both output [ 4.99721, -2.00968, 0.03363, -0.02145, 1.02895].

Paul posted this analysis in the hopes that if someone else is in the same situation trying to match resuts of ridge regression using statsmodels and sklearn, they can find the answer more easily (since Paul had not seen any discussion of this difference before). It is also noteworthy that sklearn's Ridge re-scales the tuning parameter but sklearn's Lasso does not. Paul was not able to find an explanation of this behaviour in the sklearn documentation for Ridge and LASSO.

### Variance Inflation Factors for ridge regression

Josef Perktold wrote the followng function to calculate VIF for ridge regression:



In [11]:
def vif_ridge(X, pen_factors, is_corr=False):

    """
    Variance Inflation Factor for Ridge regression 

    adapted from statsmodels function vif_ridge by Josef Perktold https://gist.github.com/josef-pkt
    source: https://github.com/statsmodels/statsmodels/issues/1669
    source: https://stackoverflow.com/questions/23660120/variance-inflation-factor-in-ridge-regression-in-python
    author: https://stackoverflow.com/users/333700/josef
    Josef is statsmodels maintainer and developer, semi-retired from scipy.stats maintainance

    assumes penalization is on standardized feature variables
    assumes alpha is scaled by n_samples in calc of penalty factors if using sklearn Ridge (see note below)
    data should not include a constant

    Parameters
    ----------
    X : array_like with dimension n_samples x n_features
        correlation matrix if is_corr=True or standardized feature data if is_corr is False.
    pen_factors : iterable array of of Ridge penalization factors with dimension n_alpha x n_coef, 
        where in a fitted Ridge model at each alpha in an iterable vector:
            pen_factor = alpha * np.sum(coefficients ** 2)
        where coefficients is the array of coefficients at each alpha
        excluding the intercept, and 
            alpha = alpha_input / n_samples (if using sklearn Ridge), or
            alpha = alpha_input (if using statsmodels ridge with OLS.fit_regularized)
        where alpha_input is the input argument of alpha used 
        with either sklearn or statsmodels, depending on whichever you are using
        (see explanation in note below for difference between sklearn and statsmodels)
    is_corr : bool
        Boolean to indicate how corr_x is interpreted, see corr_x

    Returns
    -------
    vif : ndarray
        variance inflation factors for parameters in columns and 
        ridge penalization factors in rows

    could be optimized for repeated calculations

    Note about scaling of alpha in statsmodels vs sklearn 
    -------
    An analysis by Paul Zivich (https://sph.unc.edu/adv_profile/paul-zivich/) explains 
    how to get the same results of ridge regression from statsmodels and sklearn. 
    The difference is that sklearn's Ridge function scales the input of the 'alpha' 
    regularization term during excecution as alpha / n where n is the number of observations, 
    compared with statsmodels which does not apply this scaling of the regularization 
    parameter during execution. You can have the ridge implementations match 
    if you re-scale the sklearn input alpha = alpha / n for statsmodels. 
    Note that this rescaling of alpha only applies to ridge regression. 
    The sklearn and statsmodels results for Lasso regression using exactly 
    the same alpha values for input without rescaling.
    
    Here is a link to the original post of this analysis by Paul Zivich:
    
    https://stackoverflow.com/questions/72260808/mismatch-between-statsmodels-and-sklearn-ridge-regression
    
    Therefore, since this vif_ridge function was developed for statsmodels,
    and if you are using this function with the sklearn Ridge functions,
    you will need to calculate the penalty factors as follows:

    pen_factor = alpha / n_samples * (coefficients ** 2)
    
    To scale alpha by the number of samples using sklearn, 
    you would need to do it manually, like this:
    
    from sklearn.linear_model import Ridge    
    n_samples = X.shape[0]
    scaled_alpha = alpha / n_samples
    ridge = Ridge(alpha=scaled_alpha)
    ridge.fit(X, y)

    Example calculation of pen_factors and use of vif_ridge (statsmodels function)
    -------
    from sklearn.datasets import make_regression
    X, y, w = make_regression(
        n_samples=100, n_features=10, n_informative=8, coef=True, random_state=1)
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import Ridge
    from sklearn.metrics import mean_squared_error
    clf = Ridge()
    # Generate values for `alpha` that are evenly distributed on a logarithmic scale
    n_samples = X.shape[0]
    alphas = np.logspace(-3, 4, 200)
    coefs = []
    errors_coefs = []
    pen_factors = []
    # Train the model with different regularisation strengths
    for a in alphas:
        clf.set_params(alpha=a).fit(X, y)
        coefs.append(clf.coef_)
        errors_coefs.append(mean_squared_error(clf.coef_, w))
        # Extract coefficients (w)
        coefficients = clf.coef_
        # Calculate the penalty term
        # pen_factors.append(a * np.sum(coefficients ** 2))
        # NOTE:
        # sklearn ridge scales input alpha during execution by dividing by n_samples to calculate penalty factor
        # statsmodels ridge regression does not scale input alpha by n_samples during execution
        # https://stackoverflow.com/questions/72260808/mismatch-between-statsmodels-and-sklearn-ridge-regression
        # uncomment one of the following two lines depending on if you are using sklearn or statsmodels
        pen_factors.append(a / n_samples * np.sum(clf.coef_ ** 2))     # if using sklearn for ridge regression
        # pen_factors.append(a * np.sum(clf.coef_ ** 2))           # if using statsmodels for ridge regression
    # dataframe of the series of VIF values corresponding the penalty factors at each value of alpha
    vifs = pd.DataFrame(vif_ridge(X, pen_factors))
        
    """

    import numpy as np
    
    X = np.asarray(X)
    if not is_corr:
        corr = np.corrcoef(X, rowvar=0, bias=True)
    else:
        corr = X

    eye = np.eye(corr.shape[1])
    res = []
    for k in pen_factors:
        minv = np.linalg.inv(corr + k * eye)
        vif = minv.dot(corr).dot(minv)
        res.append(np.diag(vif))

    return np.asarray(res)


# Note to Josef Perktold regarding use of vif_ridge:

Hi Josef,

For example, if I use the following input data for X and y:

```
import numpy as np
import pandas as pd 
import statsmodels.api as sm
from sklearn.linear_model import Lasso, Ridge
np.random.seed(142131)
n = 500
d = pd.DataFrame()
d['A'] = np.random.normal(size=n)
d['B'] = d['A'] + np.random.normal(scale=0.25, size=n)
d['C'] = np.random.normal(size=n)
d['D'] = np.random.normal(size=n)
d['intercept'] = 1
d['Y'] = 5 - 2*d['A'] + 1*d['D'] + np.random.normal(size=n)
y = np.asarray(d['Y'])
X = np.asarray(d[['intercept', 'A', 'B', 'C', 'D']])
```

If I want to get the same ridge regression results using sklearn and statsmodels, then I have to use different input values of alpha for each to get the same output of rigde regression coefficients as shown in the following code:

```
# use sklearn Ridge
alpha_sklearn = 1
ridge = Ridge(alpha=alpha_sklearn, fit_intercept=True)
ridge.fit(X=np.asarray(d[['A', 'B', 'C', 'D']]), y=y)
print('output params from sklearn with alpha = 1: \n',ridge.intercept_, ridge.coef_)

# use statsmodels OLS fit_regularized L1_wt=0
# NOTE: we need to re-scale the alpha parameter used in sklearn to get samee results in statsmodels
n_samples = X.shape[0]
alpha_statsmodels = np.array([0, 1., 1., 1., 1.]) / n_samples  # scaling of alpha by n
ols = sm.OLS(y, X).fit_regularized(L1_wt=0., alpha=alpha_statsmodels)
print('output params from statsmodels with alpha = 1 / n_samples: \n',ols.params)
```

This results in the following output of ridge regression coefficients from sklearn and statsmodels

```
ridge params from sklearn intercept and coefs: 
 4.997208595888691 [-2.00968258  0.03363013 -0.02144874  1.02895154]
ridge params from statsmodels with alpha=alpha/n: 
 [ 4.9972086  -2.00968258  0.03363013 -0.02144874  1.02895154]
```

In other words, if I use an input value of alpha=1 with sklearn, then I need to use an input value of alpha=1/n_samples with statsmodels if I want to get the same results for ridge regression.

Based on this behavior, it appears that the sklearn implementation of ridge regression is internally scaling the alpha parameter by n_samples. Therefore, I am thinking that if I am using your vif_ridge function with sklearn for my analysis, then the input values of pen_factors for vif_ridge should be as follows:

pen_factors = alpha / n_samples

where alpha is the input value that I use for the sklearn Ridge function, and n_samples is X.shape[0].

Does that approach look correct to you?

Best,

Greg



In [9]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
from sklearn.linear_model import Lasso, Ridge
np.random.seed(142131)
n = 500
d = pd.DataFrame()
d['A'] = np.random.normal(size=n)
d['B'] = d['A'] + np.random.normal(scale=0.25, size=n)
d['C'] = np.random.normal(size=n)
d['D'] = np.random.normal(size=n)
d['intercept'] = 1
d['Y'] = 5 - 2*d['A'] + 1*d['D'] + np.random.normal(size=n)
y = np.asarray(d['Y'])
X = np.asarray(d[['intercept', 'A', 'B', 'C', 'D']])

In [10]:
# use sklearn Ridge
alpha_sklearn = 1
ridge = Ridge(alpha=alpha_sklearn, fit_intercept=True)
ridge.fit(X=np.asarray(d[['A', 'B', 'C', 'D']]), y=y)
print('output params from sklearn with alpha = 1: \n',ridge.intercept_, ridge.coef_)

# use statsmodels OLS fit_regularized L1_wt=0
# NOTE: we need to re-scale the alpha parameter used in sklearn to get samee results in statsmodels
n_samples = X.shape[0]
alpha_statsmodels = np.array([0, 1., 1., 1., 1.]) / n_samples  # scaling of alpha by n
ols = sm.OLS(y, X).fit_regularized(L1_wt=0., alpha=alpha_statsmodels)
print('output params from statsmodels with alpha = 1 / n_samples: \n',ols.params)

output params from sklearn with alpha = 1: 
 4.997208595888691 [-2.00968258  0.03363013 -0.02144874  1.02895154]
output params from statsmodels with alpha = 1 / n_samples: 
 [ 4.9972086  -2.00968258  0.03363013 -0.02144874  1.02895154]
